# Optiver Realized Volatility Prediction (1 stock)

This competition's main objective is to predit short-term volatility for 112 stocks. In this notebook, we'll attempt to predict realized volatility for one stock and then replicate the process across all stocks.
---
## Steps:
1. Load and preprocess the data.
2. Engineer features to improve prediction accuracy.
3. Train a machine learning model (XGBoost) to predict volatility.
4. Evaluate the model using RMSPE and R-squared metrics.
5. Scale the process to handle all 112 stocks.
---

In [2]:
import pandas as pd
import numpy as np
import glob
import plotly.express as px
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from joblib import Parallel, delayed

# Set stock_id for demonstration purposes
stock_id = 0
book_path="/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet"
trade_path="/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet"

## Step 1: Load and Preprocess Data

We'll load the order book and trade data for one stock (`stock_id=0`) and filter it by `time_id=5` for demonstration purposes.

In [3]:
# Load order book data for stock_id=0, time_id=5
book_example = pd.read_parquet(f'{book_path}/stock_id=0')
book_example = book_example[book_example['time_id'] == 5]
display(book_example.head())

# Load trade data for stock_id=0, time_id=5
trade_example = pd.read_parquet(f'{trade_path}/stock_id=0')
trade_example = trade_example[trade_example['time_id'] == 5]
display(trade_example.head())

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100


,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1


## Step 2: Feature Engineering

We'll calculate Weighted Average Price (WAP), log returns, and other features that help improve predictions.

In [4]:
# Function to calculate Weighted Average Price (WAP)
def calculate_wap(bid_price1, ask_price1, bid_size1, ask_size1):
    return (bid_price1 * ask_size1 + ask_price1 * bid_size1) / (bid_size1 + ask_size1)

# Calculate WAP for book data
book_example['wap'] = calculate_wap(
    book_example['bid_price1'], 
    book_example['ask_price1'], 
    book_example['bid_size1'], 
    book_example['ask_size1']
)

# Plot WAP over time for visualization
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [5]:
# Function to calculate log returns
def log_returns(list_stock_prices):
    return np.log(list_stock_prices).diff()

# Calculate log returns for WAP
book_example['log_return'] = log_returns(book_example['wap'])
book_example = book_example[~book_example['log_return'].isnull()]

# Plot log returns over time for visualization
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

## Step 3: Calculate Realized Volatility

Realized volatility is calculated as the square root of the sum of squared log returns within a given time period.

In [6]:
# Function to calculate realized volatility
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

realized_vol = realized_volatility(book_example['log_return'])
print(f"Realized volatility for stock_id=0 on time_id=5 is {realized_vol}")

Realized volatility for stock_id=0 on time_id=5 is 0.004499364172786559


## Step 4: Scale the Process Across All Stocks

We'll now calculate realized volatility across all stocks using parallel processing to speed up computations.

In [7]:
# List all order book files for training data
list_order_book_file_train = glob.glob(f'{book_path}/*')
print(f"Number of stocks: {len(list_order_book_file_train)}")

# Sort files by stock_id for consistency
list_order_book_file_train = sorted(list_order_book_file_train, key=lambda x: int(x.split('=')[1]))

Number of stocks: 112


In [8]:
# Function to calculate realized volatility per stock and time_id
def realized_volatility_per_time_id(file_path):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] = calculate_wap(
        df_book_data['bid_price1'], 
        df_book_data['ask_price1'], 
        df_book_data['bid_size1'], 
        df_book_data['ask_size1']
    )
    df_book_data['target'] = df_book_data.groupby(['time_id'])['wap'].transform(log_returns)
    df_book_data = df_book_data[~df_book_data['target'].isnull()]
    
    # Calculate realized volatility per time_id
    df_realized_vol_per_stock = pd.DataFrame(
        df_book_data.groupby(['time_id'])['target'].agg(realized_volatility)
    ).reset_index()
    
    # Add row_id column for submission format (e.g., "stock_id-time_id")
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x: f'{stock_id}-{x}')
    
    return df_realized_vol_per_stock[['row_id', 'target']]

In [9]:
# Process all stocks in parallel using Joblib
def past_realized_volatility_per_stock(list_file):
    results = Parallel(n_jobs=-1)(delayed(realized_volatility_per_time_id)(file) for file in list_file)
    return pd.concat(results)

df_past_realized_train = past_realized_volatility_per_stock(list_order_book_file_train)
display(df_past_realized_train.head())

,row_id,target
0,0-5,0.004499
1,0-11,0.001204
2,0-16,0.002369
3,0-31,0.002574
4,0-62,0.001894


## Step 5: Train XGBoost Model

We'll use XGBoost to predict realized volatility based on features like WAP and log returns.

In [10]:
from sklearn.metrics import r2_score

# Prepare training data (example with one stock)
X = book_example[['wap', 'log_return']]
y = book_example.groupby('time_id')['log_return'].transform(realized_volatility)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.05)
xgb_model.fit(X_train, y_train)

# Make predictions and evaluate performance
y_pred = xgb_model.predict(X_test)

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(((y_true - y_pred) / y_true)**2))

print(f"RMSPE: {rmspe(y_test, y_pred):.4f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")

RMSPE: 0.0000
R²: -45352291516920.0000


## Step 6: Submission File

Combine predictions across all stocks into a single submission file.

In [11]:
submission_file = 'submission.csv'
df_past_realized_train.to_csv(submission_file, index=False)
print(f"Submission saved to {submission_file}")

Submission saved to submission.csv
